In [3]:
import gzip
import pandas as pd
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import matplotlib.pyplot as plt
import warnings
from collections import OrderedDict
warnings.filterwarnings("ignore")

from pathlib import Path

## Exploratory Data Analysis

In [5]:
#data = pd.read_csv("interactions_train.csv")
data_raw_interactions = pd.read_csv("RAW_interactions.csv")
#data_raw_interactions = data_raw_interactions.sample(50000)

In [ ]:
#size of dataset
data_raw_interactions.shape

In [ ]:
data_raw_interactions.iloc[0,:]

In [ ]:
recipeperUser = defaultdict(list)
userperRecipe  = defaultdict(list)
ratingsperUser = defaultdict(list)
ratingsperDate = defaultdict(list)
ratingsperRecipe = defaultdict(list)
occurenceperRating = defaultdict(list)
all_recipes = set()
all_users = set()
for index, row in data_raw_interactions.iterrows():
    u,i,r =  row['user_id'], row['recipe_id'],row['rating']
    recipeperUser[u].append(i)
    userperRecipe[i].append(u)
    ratingsperRecipe[i].append(r)
    ratingsperUser[u].append((i,r))
    all_recipes.add(row['recipe_id'])
    all_users.add(row['user_id'])
    ratingsperDate[row['date']].append(r)
    if r in occurenceperRating.keys():
        occurenceperRating[r] +=1
    else:
        occurenceperRating[r] = 0

In [ ]:
x = len(recipeperUser) #no of users
y = len(userperRecipe) #no of recipes
z = len(ratingsperDate) # number of dates
x,y,z

In [ ]:
#print(max(ratingsperDate, key = ratingsperDate.get))

In [ ]:
number_of_ratingsperDate = defaultdict(list)

for x in ratingsperDate.keys():
    number_of_ratingsperDate[x] = len(ratingsperDate[x])

In [ ]:
date_with_max_number_of_ratings = max(number_of_ratingsperDate, key = number_of_ratingsperDate.get)
date_with_min_number_of_ratings = min(number_of_ratingsperDate, key = number_of_ratingsperDate.get)

In [ ]:
print('Date with max # of ratings',date_with_max_number_of_ratings, number_of_ratingsperDate[date_with_max_number_of_ratings])
print('Date with min # of ratings',date_with_min_number_of_ratings, number_of_ratingsperDate[date_with_min_number_of_ratings])

In [ ]:
dates = list(ratingsperDate.keys())

values = list(ratingsperDate.values())
 
#################################################
print(min(number_of_ratingsperDate.keys()))
print(max(number_of_ratingsperDate.keys()))
print("")

binned_of_ratingsperDate = defaultdict(list)
for x in number_of_ratingsperDate.keys():
    year = x.split("-", 1)[0]
    year = int(year) - 2000
    if year in binned_of_ratingsperDate.keys(): 
        binned_of_ratingsperDate[year] += number_of_ratingsperDate[x]
    else:
        binned_of_ratingsperDate[year] = 0
    

for x in binned_of_ratingsperDate.keys():
    print(x, binned_of_ratingsperDate[x])
binned_of_ratingsperDate = OrderedDict(sorted(binned_of_ratingsperDate.items()))
################################################
values_length = [binned_of_ratingsperDate[x] for x in binned_of_ratingsperDate.keys()]
plot_x_axis = [str(x) for x in binned_of_ratingsperDate.keys()]
print (plot_x_axis)
plt.bar(plot_x_axis, values_length)
plt.xlabel("Year in 2000s")
plt.ylabel("Number of Ratings")

plt.show()

In [ ]:
binned_of_ratingsperDate_per_rating = defaultdict(list)

for x in ratingsperDate.keys():
    one = ratingsperDate[x].count(1)
    two = ratingsperDate[x].count(2)
    three = ratingsperDate[x].count(3)
    four = ratingsperDate[x].count(4)
    five = ratingsperDate[x].count(5)
    zero = ratingsperDate[x].count(0)
    year = x.split("-", 1)[0]
    year = int(year) - 2000
    if year in binned_of_ratingsperDate_per_rating.keys():
            binned_of_ratingsperDate_per_rating[year][0]+=zero
            binned_of_ratingsperDate_per_rating[year][1]+=one
            binned_of_ratingsperDate_per_rating[year][2]+=two
            binned_of_ratingsperDate_per_rating[year][3]+=three
            binned_of_ratingsperDate_per_rating[year][4]+=four
            binned_of_ratingsperDate_per_rating[year][5]+=five
            #print(binned_of_ratingsperDate_per_rating[year][0])
    else:
        binned_of_ratingsperDate_per_rating[year] = [0,0,0,0,0,0]
        
        
binned_of_ratingsperDate_per_rating = OrderedDict(sorted(binned_of_ratingsperDate_per_rating.items()))
#################################################################

zeros = [binned_of_ratingsperDate_per_rating[x][0] for x in binned_of_ratingsperDate_per_rating.keys()]
ones = [binned_of_ratingsperDate_per_rating[x][1] for x in binned_of_ratingsperDate_per_rating.keys()]
twos = [binned_of_ratingsperDate_per_rating[x][2] for x in binned_of_ratingsperDate_per_rating.keys()]
threes = [binned_of_ratingsperDate_per_rating[x][3] for x in binned_of_ratingsperDate_per_rating.keys()]
fours = [binned_of_ratingsperDate_per_rating[x][4] for x in binned_of_ratingsperDate_per_rating.keys()]
fives = [binned_of_ratingsperDate_per_rating[x][5] for x in binned_of_ratingsperDate_per_rating.keys()]
plot_x_axis = [str(x) for x in binned_of_ratingsperDate.keys()]
print (plot_x_axis)
fig0 = plt.figure()
ax0 = fig0.add_axes([0,0,1,1])
ax0.bar(plot_x_axis, zeros, color = 'b', width = 0.5, label='zeros')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 0 Ratings")

plt.show()


In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_axes([0,0,1,1])
ax1.bar([str(int(yr)) for yr in plot_x_axis], ones, color = 'g', width = 0.5, label='ones')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 1 Ratings")
plt.show()

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_axes([0,0,1,1])
ax2.bar([str(int(yr)) for yr in plot_x_axis], twos, color = 'y', width = 0.5, label='twos')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 2 Ratings")
plt.show()

In [ ]:
fig3 = plt.figure()
ax3 = fig3.add_axes([0,0,1,1])
ax3.bar([str(int(yr)) for yr in plot_x_axis], threes, color = 'r', width = 0.5, label='threes')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 3 Ratings")
plt.show()

In [ ]:
fig4 = plt.figure()
ax4 = fig4.add_axes([0,0,1,1])
ax4.bar([str(int(yr)) for yr in plot_x_axis], fours, color = 'orange', width = 0.5, label='fours')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 4 Ratings")
plt.show()

In [ ]:
fig5 = plt.figure()
ax5 = fig5.add_axes([0,0,1,1])
ax5.bar([str(int(yr)) for yr in plot_x_axis], fives, color = 'maroon', width = 0.5, label='fives')
plt.xlabel("Year in 2000s")
plt.ylabel("Number of 5 Ratings")
plt.show()

In [ ]:
rating_keys = list(occurenceperRating.keys())

rating_occurences = list(occurenceperRating.values())
plt.bar(range(len(occurenceperRating)), rating_occurences)
plt.show()

In [ ]:
## RAW_recipes parsing

In [ ]:
data_raw_recipes = pd.read_csv("RAW_recipes.csv")

In [ ]:
data_raw_recipes.shape

In [ ]:
#data_raw_recipes.head

In [ ]:
data_raw_recipes.columns

In [ ]:
data_raw_recipes.iloc[0,:]

In [ ]:
features_per_recipe = defaultdict(list)


#all_users = set()
for index, row in data_raw_recipes.iterrows():
    name,recipe_id,mins,cont_id,sub,tag,nutri,n_steps,steps,descrip,ingred,n_ingred =  row['name'], row['id'],row['minutes'],row['contributor_id'], row['submitted'], row['tags'], row['nutrition'],row['n_steps'],row['steps'], row['description'],row['ingredients'], row['n_ingredients']
    features_per_recipe[recipe_id] = [name,mins,cont_id,sub,tag,nutri,n_steps,steps,descrip,ingred,n_ingred]
    

In [ ]:
print(features_per_recipe[137739])

In [ ]:
all_tags = set()
for x in features_per_recipe.keys():
    tag = features_per_recipe[x][4]
    tag = tag.replace('[', '')
    tag = tag.replace(']', '')
    tag = tag.replace('\'', '')
    tag = tag.split(",")
    for y in tag:
        all_tags.add(y)
    #print(tag)

In [ ]:
len(all_tags)

In [ ]:
ratingsperNSteps = defaultdict(list)
for x in ratingsperRecipe:
    for y in ratingsperRecipe[x]:
        ratingsperNSteps[features_per_recipe[x][6]].append(y)


In [ ]:
avg_ratingsperNSteps = defaultdict(list)
ratingsperNSteps = OrderedDict(sorted(ratingsperNSteps.items()))
for x in ratingsperNSteps:
    avg_ratingsperNSteps[x] = [sum(ratingsperNSteps[x])/len(ratingsperNSteps[x]),len(ratingsperNSteps[x])]
    print(x,avg_ratingsperNSteps[x][0],avg_ratingsperNSteps[x][1])


In [ ]:
len(avg_ratingsperNSteps)

In [ ]:
data_pp_recipes = pd.read_csv("PP_recipes.csv")

In [ ]:
data_pp_recipes.shape

### End of code 

In [ ]:
imgCountAll = defaultdict(int)
totalliked = 0

for user,img,_ in interactions:
    imgCountAll[img] += 1
    totalliked+= 1

mostPopular = [(imgCountAll[x], x) for x in imgCountAll]
mostPopular.sort()
mostPopular.reverse()
print(totalliked)


In [ ]:
return2 = set()
count = 0
threshold = 0.75
for imgCount, img in mostPopular:
    count += imgCount
    return2.add(img)
    if count > totalliked*threshold:
        break

In [ ]:
def Jaccard(s1, s2):
  if not isinstance(s1,set):
    s1 = set(s1)
  if not isinstance(s2,set):
    s2 = set(s2)
  numer = len(s1.intersection(s2))
  denom = len(s1.union(s2))
  if denom == 0:
      return 0
  return numer / denom

def predictRead(user, img):
  Jsimilarities = []
  for d in imgsperUser[user]:
    #d is img user has liked in training
    if d == img: continue
    Jsimilarities.append(Jaccard(userperImgs[img],userperImgs[d]))
  if Jsimilarities and max(Jsimilarities)> 0.003 or img in return2:
    return 1
  return 0
  # mx = 0
  # if Jsimilarities:
  #     mx = max(Jsimilarities)
  # elif len(Jsimilarities) == 0:
  #   mx = 0
  # if img in return2:
  #     return 1 + mx
  # return mx

In [ ]:
likedValid = interactions[990000:]
likedValid = [i + (1,) for i in likedValid]


imgsperUserVal = defaultdict(list)
for u,img,_,_ in likedValid:
    imgsperUserVal[u].append(img)

len(likedValid)

In [ ]:
all_books = set(userperImgs.keys())
val_pairs = set()
for user in imgsperUserVal.keys():
  no_books_read = len(imgsperUserVal[user])
  books_not_read = list(all_books.difference(set(imgsperUser[user])))
  for _ in range(no_books_read):
    book = random.choice(books_not_read)
    if (user,book, 'NA', 0) not in val_pairs:
      book = book
    else:
      while (user,book, 'NA', 0) in val_pairs:
        book = random.choice(books_not_read)
    val_pairs.add((user,book,'NA',0))
    likedValid.append((user,book,'NA',0)) 

len(likedValid)

In [ ]:
preds = [predictRead(u,b) for u,b,_,_ in likedValid]
correct = sum([likedValid[i][3]  == preds[i] for i in range(len(likedValid))])
y = [x for _,_,_,x in likedValid]
acc4 = correct/len(preds)
acc4 